In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import zipfile
import torch

# pytorch modules
import timm # to load the torch image processing models, will use resnrt34
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.transforms as transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## unzip files

In [3]:
zip_file = "/kaggle/input/platesv2/plates.zip"
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
  zip_ref.extractall()  # extract current working directory

## training config

In [4]:
seed = 42
batch_size = 16
num_epochs = 5

In [5]:
transformation = transforms.Compose(
            [
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ])

## data

In [6]:
data_dir = "/kaggle/working/plates"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"

In [7]:
os.listdir(train_dir)

['dirty', 'cleaned', '.DS_Store']

In [8]:
# not sure why we have .DS_Store
labels = ['dirty', 'cleaned']

In [9]:
from torchvision import datasets, models
from torch.utils.data import DataLoader

In [10]:
# gets images from all subfolders as well
train_data = datasets.ImageFolder(train_dir, transform=transformation)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
train_data, train_loader

(Dataset ImageFolder
     Number of datapoints: 40
     Root location: /kaggle/working/plates/train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 <torch.utils.data.dataloader.DataLoader at 0x7b3f4cad2b00>)

In [11]:
class_mapping = train_data.class_to_idx
class_mapping

{'cleaned': 0, 'dirty': 1}

## training

In [12]:
from tqdm.auto import tqdm

In [23]:
def train_model(model, optimizer, criterion, train_loader, epochs):
    plot_training = []

    for e in tqdm(range(epochs)):
        model.train()
        epoch_loss = []

        for images, labels in iter(train_loader):
            # reset gradient calculation
            optimizer.zero_grad()
            
            # forward pass
            images = images.to(device)
            output = model.forward(images)
            # loss calculation
            loss = criterion(output, labels.to(device))
            
            # backward pass
            loss.backward()
            # update weights
            optimizer.step()

            epoch_loss.append(loss.item())
            
        print(f"Epoch {e+1}/{epochs}: Loss {sum(epoch_loss)/len(epoch_loss)}")

    return model

In [14]:
model = models.resnet34(weights=True)
print(model)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 164MB/s] 


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
# freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
    parameter.requires_grad = False

In [16]:
from collections import OrderedDict
classifier = nn.Sequential(
    OrderedDict(
        [
            ("fc", nn.Linear(512, 2)),
            ("output", nn.LogSoftmax(dim=1)),
        ]
    )
)
model.fc = classifier

In [17]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [18]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [24]:
model = train_model(model, optimizer, criterion, train_loader, 100)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100: Loss 0.6711268027623495
Epoch 2/100: Loss 0.6158392031987509
Epoch 3/100: Loss 0.5922025640805563
Epoch 4/100: Loss 0.5450810790061951
Epoch 5/100: Loss 0.43776610493659973
Epoch 6/100: Loss 0.42528138558069867
Epoch 7/100: Loss 0.4179363747437795
Epoch 8/100: Loss 0.40620556473731995
Epoch 9/100: Loss 0.35284581780433655
Epoch 10/100: Loss 0.3224724729855855
Epoch 11/100: Loss 0.3394774794578552
Epoch 12/100: Loss 0.3136350413163503
Epoch 13/100: Loss 0.3146231969197591
Epoch 14/100: Loss 0.5055507322152456
Epoch 15/100: Loss 0.29483048617839813
Epoch 16/100: Loss 0.2997368574142456
Epoch 17/100: Loss 0.2752589633067449
Epoch 18/100: Loss 0.26631322006384534
Epoch 19/100: Loss 0.26871900757153827
Epoch 20/100: Loss 0.20596520602703094
Epoch 21/100: Loss 0.22909259299437204
Epoch 22/100: Loss 0.2573130925496419
Epoch 23/100: Loss 0.2583098163207372
Epoch 24/100: Loss 0.2988799462715785
Epoch 25/100: Loss 0.21606635053952536
Epoch 26/100: Loss 0.2180156707763672
Epoch 27/10

## eval

In [25]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
from torch.utils.data import Dataset
from PIL import Image
class CustomTestDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = [f for f in os.listdir(img_dir) if f != ".DS_Store" and os.path.isfile(os.path.join(img_dir, f))]
    
    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image


In [27]:
test_dataset = CustomTestDataset(img_dir=test_dir, transform=transformation)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

In [45]:
def get_predicted_class(output_tensor):
    probabilities = torch.exp(output_tensor)
    predicted_class = torch.argmax(output_tensor).item()
    
    return "dirty" if predicted_class == 0 else "cleaned"

In [46]:
classes = []
test = None
for image in test_loader:
    preds = model(image.to(device))
    for pred in preds:
        temp = get_predicted_class(pred)
        classes.append(temp)

In [47]:
import pandas as pd
submission_df = pd.read_csv('/kaggle/input/platesv2/sample_submission.csv')
submission_df['label'] = classes
submission_df.to_csv("/kaggle/working/submission.csv", index=False)